<a href="https://colab.research.google.com/github/erbisti/Collab/blob/main/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt

# ===== 1. Carregamento e preparação dos dados =====
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.
x_train = np.reshape(x_train, (-1, 28, 28, 1))
x_test = np.reshape(x_test, (-1, 28, 28, 1))

# ===== 2. Parâmetros =====
latent_dim = 2

# ===== 3. Encoder =====
encoder_inputs = tf.keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, strides=2, activation='relu', padding='same')(encoder_inputs)
x = layers.Conv2D(64, 3, strides=2, activation='relu', padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, name='z')([z_mean, z_log_var])
encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')

# ===== 4. Decoder =====
latent_inputs = tf.keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation='relu')(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, strides=2, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(32, 3, strides=2, activation='relu', padding='same')(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)
decoder = Model(latent_inputs, decoder_outputs, name='decoder')

# ===== 5. Modelo VAE com cálculo manual da perda =====
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        _, _, z = self.encoder(inputs)
        return self.decoder(z)

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]

        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)

            reconstruction_loss = tf.reduce_mean(
                tf.keras.losses.binary_crossentropy(data, reconstruction)
            ) * 28 * 28

            kl_loss = -0.5 * tf.reduce_mean(
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            )

            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

# ===== 6. Instanciamento e treinamento =====
vae = VAE(encoder, decoder)
vae.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())  # Dummy loss
vae.fit(x_train, x_train,
        epochs=10,
        batch_size=128,
        validation_data=(x_test, x_test))

# ===== 7. Visualização das reconstruções =====
decoded_imgs = vae.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    ax.axis('off')
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    ax.axis('off')
plt.show()


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 100s 202ms/step - kl_loss: 3.2470 - loss: 211.0954 - reconstruction_loss: 207.8484 - val_loss: 0.0562
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 133s 183ms/step - kl_loss: 3.8646 - loss: 171.3172 - reconstruction_loss: 167.4526 - val_loss: 0.0499
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 87s 185ms/step - kl_loss: 3.7628 - loss: 163.1036 - reconstruction_loss: 159.3408 - val_loss: 0.0470
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 138s 178ms/step - kl_loss: 3.8113 - loss: 159.3088 - reconstruction_loss: 155.4975 - val_loss: 0.0458
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 142s 178ms/step - kl_loss: 3.8246 - loss: 157.1450 - reconstruction_loss: 153.3204 - val_loss: 0.0450
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 145s 184ms/step - kl_loss: 3.8700 - loss: 155.6795 - reconstruction_loss: 151.8095 - val_loss: 0.0442
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 140s 180ms/step - kl_loss: 3.8636 - loss: 154.4384 - reconstruction_loss: 150.5748 - val_loss: 0.0439
Epoch 8